# Multilingual Machine Learning
### Exploring BLEU Scores using Patent Data
#### by Lee Mackey

This notebook accompanies the article [Multilingual Machine Learning](https://medium.com/@glmack)
<br>

<br>
## Introduction
---

Does your machine learn in Chinese? I don't speak Mandarin or Cantonese so Google Translate gets all the credit \- good or bad \- for translating the preceding sentence into: "您和您的機器學習中文嗎?" But how might a researcher quickly evaluate the quality of machine translations? This question encapsulates the basic challenge that gives rise to the BLEU metric. BLEU, which stands for bilingual language understudy, is a default measure of machine translation quality and is also sometimes applied to [cross-lingual](https://arxiv.org/abs/1901.07291) natural language processing (NLP) tasks. The metric is well-established in the machine translation space but some analysts question the algorithm's applicability to a wider set of tasks beyond the measure's original objectives. This article takes an initial dive into the lessons, implementations and limits of BLEU using examples drawn from multilingual patent documents.

In [ ]:
# import modules
from multilingual_machines import (split_tokens, 
                                   clean_punctuation,
                                   represent_references)
from nltk.translate.bleu_score import corpus_bleu
from nltk import bigrams, trigrams, ngrams
import string
import json
from IPython.display import IFrame

In [ ]:
# load example data from 'patent_examples.txt' file in Github repo
with open('patent_examples.txt') as f:
    data = json.load(f)

<br>
## Basics of BLEU
---

Researchers at IBM developed the BLEU algorithm in 2002 as an efficient method to evaluate the quality of machine translation in reference to benchmark human translations. The original [paper](https://www.aclweb.org/anthology/P02-1040.pdf) by the developers, Papineni and colleagues, is a good place to start if you’re interested in the founding context and objectives of the algorithm.

In [ ]:
# browse original BLEU paper
IFrame('https://www.aclweb.org/anthology/P02-1040.pdf', width=650, height=300)

BLEU is an adjusted precision measure of the matching word sequences between a “candidate” machine translation and one or multiple “reference” human translations. BLEU counts "n-grams", a term for word sequences of length *n*, in a machine translation that match the n-grams in a human translation, divided by the total count of n-grams in the machine translation. The measure is adjusted in that it clips the matches to the maximum count of n-gram occurrences in a human translation and also penalizes machine translations that diverge in word length from the reference translation.

The resulting BLEU score is a number between 0 and 1, in which 0 represents zero n-gram matches between candidate and reference texts and 1 might equal a machine translation that is exactly similar to one of the references. In practice, the measure counts matches across multiple word sequence lengths \- 4-grams (four-word sequences), tri-grams (three-word sequences), bi-grams (two-word sequences), and uni-grams (one-word sequences) \- via a geometric mean of the respective n-gram calculations. The algorithm was designed for comparisons at the level of a corpus of sentences, with n-gram matches calculated at the basic unit of a sentence and then combined into a corpus-level score. To clarify terminology, the use of the term "document" in the present article refers to a corpus of sentences. If you're interested in additional resources to understand the algorithm, you might check out the [video tutorial](https://www.youtube.com/watch?v=DejHQYAGb7Q) at deeplearning.ai that discusses the details of the algorithm or the [tutorial](https://machinelearningmastery.com/calculate-bleu-score-for-text-python/) at machinelearningmastery.com that explores the NLTK implementation.  To explore the use cases of the metric in a tangible way, I next apply BLEU using translations of Chinese-language patents.

## Applying BLEU to Patent Texts
---

A growing share of patents in the machine learning space are written and filed in Chinese according to a recent [report](https://www.wipo.int/publications/en/details.jsp?id=4386) by WIPO (World Intellectual Property Organization), the global organization that governs patents. To explore the basics of BLEU in this multilingual space, you can first begin with a domestic Chinese patent of an NLP innovation that the e-commerce company Alibaba extended to global coverage. The title of the Chinese language patent is displayed below.


#### Acquire translations

In [ ]:
# inspect title of example patent in Chinese language
print(data['original_title_cn'])

For additional details surrounding the example patent, WIPO's data query tool provides both [English](https://patentscope.wipo.int/search/en/detail.jsf?docId=WO2019085779) and [Chinese](https://patentscope.wipo.int/search/zh/detail.jsf?docId=WO2019085779) language versions that you can inspect in your browser.

In [ ]:
# inspect Chinese language version of patent using WIPO GUI
IFrame('https://patentscope.wipo.int/search/zh/detail.jsf?docId=WO2019085779',
       width=650,
       height=300)

# exchange url or paste in browser to inspect English version using WIPO GUI
# https://patentscope.wipo.int/search/en/detail.jsf?docId=WO2019085779

Human specialists often produce translations of equivalent quality that nonetheless differ in word choice and structure. BLEU, therefore, accepts single or multiple human translations as the references for comparison. Next, I obtain two Chinese-to-English translations of the Chinese language patent from two different human translators via [Gengo](https://gengo.com/), a web-based platform for human translators.

In [ ]:
# inspect human #1's Ch-to-En translation of patent summary
reference_human1_summary = data['reference_human1_summary']
print(reference_human1_summary)

In [ ]:
# inspect human #2's Ch-to-En translation of patent
reference_human2_summary = data['reference_human2_summary']
print(reference_human2_summary)

Next, you can source "candidate" machine translations from two separate machine learning algorithms: Google Translate and the World Intellectual Property Organization (WIPO), which are displayed below.

In [ ]:
# inspect Google Translate's Ch-to-En machine translation of patent
candidate_google_summary = data['candidate_google_summary']
print(candidate_google_summary)

In [ ]:
# inspect WIPO's Ch-to-En machine translation by WIPO of patent
candidate_wipo_summary = data['candidate_wipo_summary']
print(candidate_wipo_summary)

#### Calculate BLEU scores

There are multiple implementations and extensions of BLEU, such as the popular [sacreBLEU](https://github.com/mjpost/sacreBLEU) package, among others. The present example begins by calculating scores using the [bleu_score module](https://www.nltk.org/api/nltk.translate.html?highlight=bleu#module-nltk.translate.bleu_score) in the Natural Language Toolkit (NLTK), which is based on the original implementation by Papineni et al. The `corpus_bleu` function accepts reference and candidate translations represented as individual word tokens that you can create through standard NLP pre-processing steps.

In [ ]:
# split candidate and reference translations into tokens
ref_human1_summary = split_tokens(reference_human1_summary)
ref_human2_summary = split_tokens(reference_human2_summary)
can_google_summary = split_tokens(candidate_google_summary)
can_wipo_summary = split_tokens(candidate_wipo_summary)

# clean punctuation from tokens in candidates and references
ref_human1_summary = clean_punctuation(ref_human1_summary)
ref_human2_summary = clean_punctuation(ref_human2_summary)
can_google_summary = clean_punctuation(can_google_summary)
can_wipo_summary = clean_punctuation(can_wipo_summary)

For a document with multiple sentences, the original BLEU implementation computes the n-gram matches sentence by sentence, then sums the clipped n-gram counts for all the candidate sentences and, lastly, divides by the number of candidate n-grams in the document. As the NLTK documentation states: the original BLEU metric "calculates the micro-average precision (i.e. summing the numerators and denominators for each hypothesis-reference(s) pairs before the division)." By default, `corpus_bleu` calculates the geometric mean of equally-weighted n-gram scores comprising 4-grams, tri-grams, bi-grams and unigrams.

In [ ]:
# inspect bi-gram examples from string human #1's translation
bi_grams = list(ngrams(ref_human1_summary[0], 2))[0:10]

# inspect 4-gram examples from string of human #1's translation
four_grams = list(ngrams(ref_human1_summary[0], 4))[0:10]

print(f"bi-gram tuples: {bi_grams}", '\n')
print(f"4-grams tuples: {four_grams}")

What are the results of the BLEU score calculations for the two machine translation examples?

In [ ]:
# Compare each sentence in the Google and WIPO translations against all sentences in both references

# represent references to 6 sentence-length WIPO translation
refs_list_6 = represent_references(ref_human1_summary, ref_human2_summary, 6)

# represent references to 5 sentence-length WIPO translation
refs_list_5 = represent_references(ref_human1_summary, ref_human2_summary, 5)

The results indicate that the Google and WIPO translations are of similar quality in relation to the reference translations from human translators on Gengo. The score of the first candidate translation by Google Translate is 0.53 and the score of WIPO is 0.54, as displayed below.

In [ ]:
# calculate BLEU score of Google translation
bleu_google = round(corpus_bleu(refs_list_6, can_google_summary[:]), 2)

# calculate BLEU score of WIPO translation
bleu_wipo = round(corpus_bleu(refs_list_5, can_wipo_summary[:]), 2)

print(f"Google Translate BLEU score: {bleu_google}")
print(f"WIPO BLEU score: {bleu_wipo}")

With this tangible example of the application of BLEU in mind, it is helpful to establish the potential scope of the algorithm's applicability to machine translation and natural language processing tasks.

## Limits of BLEU
---

There is a general consensus among researchers that BLEU is expedient for evaluations of machine translation systems. However, some researchers caution that BLEU may not be appropriate for certain aspects of machine translation or wider NLP tasks. A key critique is that BLEU scores may not necessarily correlate well with significant quality differences in human translations. As Callison-Burch and colleagues argue in a highly-cited [paper](https://www.aclweb.org/anthology/E06-1032/): "there are instances when an improvement in BLEU is not sufficient to reflect a genuine improvement in translation quality, and in other circumstances that it is not necessary to improve BLEU in order to achieve a noticeable improvement in translation quality." These authors also suggest that BLEU may not be appropriate for comparing machine translation systems that utilize different techniques, detecting aspects of translation that BLEU doesn't model, or evaluating infrequent improvements within a test corpus.

Researchers that conduct a literature review of BLEU argue in a recent [paper](https://www.mitpressjournals.org/doi/full/10.1162/COLI_a_00322) that BLEU–human correlations may vary with the details of the systems being evaluated, the particular corpus texts used, and the exact protocol utilized for human evaluations. The authors, Reiter and colleagues, suggest that BLEU should be evaluated by real-world human evaluations such as A/B tests to determine contexts in which BLEU reliably correlates with real-world effectiveness. Based on the results of the literature review, these authors caution that researchers should approach BLEU as a diagnostic for machine translation at the system level but not as an evaluation technique to measure the output of a system. With the literature suggesting a general recognition of both the utility and shortcomings of the measure, some researchers, such as the author of a recent [paper](https://arxiv.org/abs/1804.08771), focus on increased clarity in the reporting of BLEU scores. This would include, for example, having researchers include preprocessing schemes and other standardized conventions for reporting the implementation and scoring details of BLEU scores. The points raised in these papers sound notes in caution in applying and interpreting BLEU scores in the patent space and across different use cases.

## Conclusion
---
This article outlines objectives, examples, and limits of BLEU scores. To run the full Python code that accompanies this article in a Jupyter notebook, you can head to the Github [repository](https://github.com/glmack/multilingual_machines) for Multilingual Machine Learning. For more background on my interest in this space, check out my website at [lee-mackey.com](http://lee-mackey.com). As patent documents in the machine learning space continue to become increasingly multilingual, particularly from Chinese language patents, the points raised in this article hopefully help you to surface and contextualize considerations when applying this algorithm to your particular use cases. If you're working across languages in your NLP workflows, understanding these details of BLEU will help to select if, when, and how to use this metric in your projects.